MULTIPLE MODLES WITH TEST RESULTS


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
import itertools

# ---------------------------
# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# ---------------------------
# Create a new output directory
output_dir = "final_multimodel"
os.makedirs(output_dir, exist_ok=True)

# ---------------------------
# Define dataset paths
# We have one training directory (which will be split for validation)
# and one test directory
train_dir = '/content/drive/MyDrive/skin_cancer_data/train'
test_dir  = '/content/drive/MyDrive/skin_cancer_data/test'

# ---------------------------
# Image sizes for each model
IMG_SIZES = {
    'InceptionV3': (299, 299),
    'ResNet50': (224, 224),
    'VGG16': (224, 224),
    'EfficientNetB0': (224, 224)
}

# ---------------------------
# Import model architectures and their respective preprocessing functions
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input as inception_preprocess
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input as resnet_preprocess
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as vgg_preprocess
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input as efficientnet_preprocess

# Define model configurations in a dictionary
MODEL_CONFIGS = {
    'InceptionV3': {
        'constructor': InceptionV3,
        'preprocess': inception_preprocess,
        'img_size': IMG_SIZES['InceptionV3']
    },
    'ResNet50': {
        'constructor': ResNet50,
        'preprocess': resnet_preprocess,
        'img_size': IMG_SIZES['ResNet50']
    },
    'VGG16': {
        'constructor': VGG16,
        'preprocess': vgg_preprocess,
        'img_size': IMG_SIZES['VGG16']
    },
    'EfficientNetB0': {
        'constructor': EfficientNetB0,
        'preprocess': efficientnet_preprocess,
        'img_size': IMG_SIZES['EfficientNetB0']
    }
}

# ---------------------------
# Data augmentation and validation split for training
train_datagen = ImageDataGenerator(
    validation_split=0.2,  # Use 20% of training data for validation
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=(0.8, 1.2),
    horizontal_flip=True,
    fill_mode='nearest'
)

# For testing, used minimal preprocessing
test_val_datagen = ImageDataGenerator()

def get_data_generators(img_size, batch_size=32):
    target_size = img_size
    # Training generator using validation_split
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary',  # Assumes two classes: "Benign" and "Malignant"
        subset='training',
        shuffle=True
    )

    # Validation generator from the same training data
    val_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='validation',
        shuffle=False
    )

    # Test generator remains as before
    test_generator = test_val_datagen.flow_from_directory(
        test_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary',
        shuffle=False
    )

    return train_generator, val_generator, test_generator

# ---------------------------
# Build model with a Lambda layer to apply model-specific preprocessing
def build_model(model_name, config):
    img_size = config['img_size']
    preprocess_func = config['preprocess']
    base_model = config['constructor'](weights='imagenet', include_top=False, input_shape=img_size + (3,))

    base_model.trainable = False  # Freeze base model

    inputs = Input(shape=img_size + (3,))
    x = Lambda(lambda img: preprocess_func(img))(inputs)
    x = base_model(x, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x)  # Binary classification

    model = Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# ---------------------------
# Utility function: Plot and save confusion matrix
def plot_confusion_matrix(cm, classes, model_name, save_path=None,
                          normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(5, 4))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(f'{model_name} - {title}')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path)
    plt.show()

# ---------------------------
# Utility function: Plot and save sample predictions for first 10 images
def plot_sample_predictions(model, test_generator, class_labels, model_name, num_images=10, save_path=None):
    test_generator.reset()
    x_test, y_test = next(test_generator)
    preds = model.predict(x_test)
    preds_labels = (preds > 0.5).astype("int32").flatten()

    plt.figure(figsize=(15, 5))
    for i in range(min(num_images, len(x_test))):
        plt.subplot(2, num_images // 2, i+1)
        plt.imshow(x_test[i].astype("uint8"))
        true_label = class_labels[int(y_test[i])]
        pred_label = class_labels[preds_labels[i]]
        plt.title(f"True: {true_label}\nPred: {pred_label}")
        plt.axis("off")
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path)
    plt.show()

# ---------------------------
# Main loop: Train and evaluate each model
results = {}

for model_name, config in MODEL_CONFIGS.items():
    print(f"\n{'='*20}\nTraining model: {model_name}\n{'='*20}")

    # Get data generators for the model-specific image size (with validation split)
    train_gen, val_gen, test_gen = get_data_generators(config['img_size'], batch_size=32)

    # Build and compile the model
    model = build_model(model_name, config)
    model.summary()

    # Early stopping callback to optimize training
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Train the model (adjust epochs as needed)
    history = model.fit(
        train_gen,
        epochs=30,
        validation_data=val_gen,
        callbacks=[early_stop],
        verbose=1
    )

    # Evaluate on the test set
    test_gen.reset()
    predictions = model.predict(test_gen, verbose=1)
    pred_labels = (predictions > 0.5).astype("int32").flatten()
    true_labels = test_gen.classes

    # Generate classification report
    class_indices = test_gen.class_indices
    class_labels = list(class_indices.keys())
    report = classification_report(true_labels, pred_labels, target_names=class_labels)
    print(f"\nClassification Report for {model_name}:\n{report}")

    # Save classification report to file
    report_path = os.path.join(output_dir, f"{model_name}_classification_report.txt")
    with open(report_path, "w") as f:
        f.write(report)

    # Plot and save confusion matrix
    cm = confusion_matrix(true_labels, pred_labels)
    cm_save_path = os.path.join(output_dir, f"{model_name}_confusion_matrix.png")
    plot_confusion_matrix(cm, classes=class_labels, model_name=model_name, save_path=cm_save_path)

    # Plot and save sample predictions (first 10 images)
    sample_pred_path = os.path.join(output_dir, f"{model_name}_sample_predictions.png")
    print(f"Sample predictions for {model_name}:")
    plot_sample_predictions(model, test_gen, class_labels, model_name, num_images=10, save_path=sample_pred_path)

    # Save history and results in the results dictionary
    results[model_name] = {
        'history': history.history,
        'classification_report': report,
        'confusion_matrix': cm
    }

    # Optionally, save the trained model as well
    model_save_path = os.path.join(output_dir, f"{model_name}_model.h5")
    model.save(model_save_path)

print("\nTraining and evaluation complete for all models. All outputs are saved in the directory:", output_dir)


Output hidden; open in https://colab.research.google.com to view.

In [1]:
# Install git (usually pre-installed in Colab)
!apt-get install -y git

# Configure git
!git config --global user.name "alekhyaparimala09"
!git config --global user.email "alekhyaparimala09@gmail.com"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [ ]:
import shutil

# Move the downloaded notebook file into the cloned repository folder
shutil.move("Multiple models_Pro.ipynb", "/content/your-repository/your_notebook_name.ipynb")


In [2]:
!git clone https://github.com/alekhyaparimala09/Skin-Cancer-Classification.git


Cloning into 'Skin-Cancer-Classification'...


In [4]:
import shutil

# Copy the notebook into the cloned repository folder (instead of moving)
shutil.copy("/content/drive/MyDrive/Colab Notebooks/Skin Cancer Classification_Pro.ipynbSkin Cancer Classification_Pro.ipynb", "/content/Skin-Cancer-Classification/Skin Cancer Classification_Pro.ipynb")



FileNotFoundError: [Errno 2] No such file or directory: 'Skin Cancer Classification_Pro.ipynb'

In [5]:
pwd

'/content'